In [3]:
# ============================================
# CELL 1: Check GPU & Install Dependencies
# ============================================
import subprocess
import sys

print("="*70)
print("🔧 CHECKING SYSTEM & INSTALLING DEPENDENCIES")
print("="*70)

# Install required packages
packages = ['torch', 'torchvision', 'timm', 'tqdm', 'pillow', 'matplotlib', 'numpy', 'pandas']
for pkg in packages:
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"🔧 CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"🔢 CUDA Version: {torch.version.cuda}")
    device = torch.device('cuda')
else:
    print("⚠️ No GPU found - will use CPU (slower)")
    device = torch.device('cpu')

print(f"\n🖥️ Using device: {device}")
print("="*70)

🔧 CHECKING SYSTEM & INSTALLING DEPENDENCIES


d:\kisaan madadgaar\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Installing pillow...

✅ PyTorch version: 2.7.1+cu118
🔧 CUDA available: True
🎮 GPU: Quadro M1200
💾 GPU Memory: 4.3 GB
🔢 CUDA Version: 11.8

🖥️ Using device: cuda


In [4]:
# ============================================
# CELL 2: Import Libraries & Setup
# ============================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import json
from tqdm import tqdm
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import timm

# ⚡ Speed optimizations
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    from torch.cuda.amp import GradScaler, autocast
    scaler = GradScaler()
    USE_AMP = True
    print("⚡ Mixed Precision (AMP) enabled for faster training!")
else:
    USE_AMP = False
    print("⚠️ AMP disabled (no GPU)")

print(f"\n✅ All imports successful!")
print(f"🖥️ Device: {device}")

⚡ Mixed Precision (AMP) enabled for faster training!

✅ All imports successful!
🖥️ Device: cuda


In [14]:
# ============================================
# CELL 3: Load Dataset from Local Path
# ============================================
print("="*70)
print("📂 LOADING LOCAL DATASET")
print("="*70)

# Dataset path - update this if your data is elsewhere
DATA_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\data\downloads\PakistanCrops_Merged")

if not DATA_DIR.exists():
    # Try alternate paths
    alt_paths = [
        Path("./data/downloads/PakistanCrops_Merged"),
        Path("../data/downloads/PakistanCrops_Merged"),
        Path("data/PakistanCrops_Merged"),
    ]
    for alt in alt_paths:
        if alt.exists():
            DATA_DIR = alt
            break

print(f"📁 Data directory: {DATA_DIR}")
print(f"   Exists: {DATA_DIR.exists()}")

if not DATA_DIR.exists():
    raise FileNotFoundError(f"❌ Data not found at {DATA_DIR}")

# Count classes and images
class_folders = [f for f in DATA_DIR.iterdir() if f.is_dir()]
print(f"\n📊 Found {len(class_folders)} classes:")

all_classes = {}
total_images = 0

for folder in sorted(class_folders):
    images = list(folder.glob('*.jpg')) + list(folder.glob('*.jpeg')) + list(folder.glob('*.png'))
    images += list(folder.glob('*.JPG')) + list(folder.glob('*.JPEG')) + list(folder.glob('*.PNG'))
    if len(images) > 0:
        all_classes[folder.name] = [str(img) for img in images]
        total_images += len(images)
        print(f"   📂 {folder.name}: {len(images):,} images")

print(f"\n✅ Total: {len(all_classes)} classes, {total_images:,} images")

📂 LOADING LOCAL DATASET
📁 Data directory: D:\kisaan madadgaar\Plant-Disease-Detection\data\downloads\PakistanCrops_Merged
   Exists: True

📊 Found 34 classes:
   📂 Cotton___diseased_cotton_leaf: 2,842 images
   📂 Cotton___diseased_cotton_plant: 7,362 images
   📂 Cotton___fresh_cotton_leaf: 4,146 images
   📂 Cotton___fresh_cotton_plant: 4,106 images
   📂 Mango___Anthracnose: 3,994 images
   📂 Mango___Bacterial_Canker: 3,994 images
   📂 Mango___Cutting_Weevil: 3,994 images
   📂 Mango___Die_Back: 3,994 images
   📂 Mango___Gall_Midge: 3,994 images
   📂 Mango___Healthy: 3,994 images
   📂 Mango___Powdery_Mildew: 3,994 images
   📂 Mango___Sooty_Mould: 3,994 images
   📂 Plantvillage___Pepper__bell___Bacterial_spot: 15,946 images
   📂 Plantvillage___Pepper__bell___healthy: 23,646 images
   📂 Plantvillage___Potato___Early_blight: 15,994 images
   📂 Plantvillage___Potato___healthy: 2,426 images
   📂 Plantvillage___Potato___Late_blight: 15,994 images
   📂 Plantvillage___Tomato__Target_Spot: 22,458

In [15]:
# ============================================
# CELL 4: Create Dataset Class
# ============================================
print("="*70)
print("🔧 CREATING DATASET")
print("="*70)

class PlantDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            image = Image.open(self.image_paths[idx]).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.labels[idx]
        except Exception as e:
            # Return a black image if loading fails
            print(f"⚠️ Error loading {self.image_paths[idx]}: {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
            if self.transform:
                image = self.transform(image)
            return image, self.labels[idx]

# Build dataset
all_image_paths = []
all_labels = []
class_names = []

for idx, (class_name, paths) in enumerate(sorted(all_classes.items())):
    class_names.append(class_name)
    all_image_paths.extend(paths)
    all_labels.extend([idx] * len(paths))

num_classes = len(class_names)
print(f"✅ Dataset created: {num_classes} classes, {len(all_image_paths):,} images")

# Show classes
print(f"\n📋 Classes:")
for i, name in enumerate(class_names):
    count = sum(1 for label in all_labels if label == i)
    print(f"   {i+1:2d}. {name}: {count:,} images")

🔧 CREATING DATASET
✅ Dataset created: 34 classes, 387,940 images

📋 Classes:
    1. Cotton___diseased_cotton_leaf: 2,842 images
    2. Cotton___diseased_cotton_plant: 7,362 images
    3. Cotton___fresh_cotton_leaf: 4,146 images
    4. Cotton___fresh_cotton_plant: 4,106 images
    5. Mango___Anthracnose: 3,994 images
    6. Mango___Bacterial_Canker: 3,994 images
    7. Mango___Cutting_Weevil: 3,994 images
    8. Mango___Die_Back: 3,994 images
    9. Mango___Gall_Midge: 3,994 images
   10. Mango___Healthy: 3,994 images
   11. Mango___Powdery_Mildew: 3,994 images
   12. Mango___Sooty_Mould: 3,994 images
   13. Plantvillage___Pepper__bell___Bacterial_spot: 15,946 images
   14. Plantvillage___Pepper__bell___healthy: 23,646 images
   15. Plantvillage___Potato___Early_blight: 15,994 images
   16. Plantvillage___Potato___Late_blight: 15,994 images
   17. Plantvillage___Potato___healthy: 2,426 images
   18. Plantvillage___Tomato_Bacterial_spot: 26,502 images
   19. Plantvillage___Tomato_Early_b

In [16]:
# ============================================
# CELL 5: Data Loaders
# ============================================
print("="*70)
print("📊 CREATING DATA LOADERS")
print("="*70)

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create and split dataset
full_dataset = PlantDiseaseDataset(all_image_paths, all_labels, train_transform)

train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Batch size - adjust based on your GPU memory
# RTX 3060 (12GB): batch_size=32-64
# RTX 3070/3080: batch_size=64-128
# GTX 1650/1660: batch_size=16-32
# CPU only: batch_size=8-16

if torch.cuda.is_available():
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_mem >= 8:
        batch_size = 32
    elif gpu_mem >= 4:
        batch_size = 16
    else:
        batch_size = 8
else:
    batch_size = 8

print(f"\n⚡ Batch size: {batch_size} (based on {gpu_mem:.1f}GB GPU memory)" if torch.cuda.is_available() else f"\n⚡ Batch size: {batch_size} (CPU mode)")

# Determine number of workers
num_workers = min(4, os.cpu_count() or 1)

train_loader = DataLoader(
    train_ds, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=torch.cuda.is_available()
)
val_loader = DataLoader(
    val_ds, batch_size=batch_size*2, shuffle=False,
    num_workers=num_workers, pin_memory=torch.cuda.is_available()
)
test_loader = DataLoader(
    test_ds, batch_size=batch_size*2, shuffle=False,
    num_workers=num_workers, pin_memory=torch.cuda.is_available()
)

print(f"\n📊 Dataset Split:")
print(f"   Training:   {len(train_ds):,} images ({len(train_loader)} batches)")
print(f"   Validation: {len(val_ds):,} images ({len(val_loader)} batches)")
print(f"   Test:       {len(test_ds):,} images ({len(test_loader)} batches)")
print(f"\n⚡ Workers: {num_workers}, Pin Memory: {torch.cuda.is_available()}")

📊 CREATING DATA LOADERS

⚡ Batch size: 16 (based on 4.3GB GPU memory)

📊 Dataset Split:
   Training:   271,558 images (16973 batches)
   Validation: 58,191 images (1819 batches)
   Test:       58,191 images (1819 batches)

⚡ Workers: 4, Pin Memory: True


In [17]:
# ============================================
# 🔴 REDUCE DATA + SMALL BATCH (4GB GPU Fix)
# ============================================
import random
import gc
random.seed(42)

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

print("="*70)
print("🔄 PREPARING DATA FOR 4GB GPU")
print("="*70)

# Sample 10% from each class
sampled_paths = []
sampled_labels = []

for class_idx, class_name in enumerate(class_names):
    class_indices = [i for i, l in enumerate(all_labels) if l == class_idx]
    sample_size = max(30, int(len(class_indices) * 0.10))
    sample_size = min(sample_size, len(class_indices))
    sampled_indices = random.sample(class_indices, sample_size)
    for i in sampled_indices:
        sampled_paths.append(all_image_paths[i])
        sampled_labels.append(all_labels[i])

all_image_paths = sampled_paths
all_labels = sampled_labels

print(f"✅ Data: {len(all_image_paths):,} images (10% sample)")

# Recreate dataset
full_dataset = PlantDiseaseDataset(all_image_paths, all_labels, train_transform)

train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# SMALL BATCH for 4GB GPU
batch_size = 8  # Very small batch for 4GB GPU
num_workers = 0  # Windows fix

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print(f"\n📊 Dataset Split:")
print(f"   Training:   {len(train_ds):,} images ({len(train_loader)} batches)")
print(f"   Validation: {len(val_ds):,} images ({len(val_loader)} batches)")  
print(f"   Test:       {len(test_ds):,} images ({len(test_loader)} batches)")
print(f"\n⚡ Batch: {batch_size}, Workers: {num_workers}")
print("✅ Ready for training!")

🔄 PREPARING DATA FOR 4GB GPU
✅ Data: 38,779 images (10% sample)

📊 Dataset Split:
   Training:   27,145 images (3394 batches)
   Validation: 5,816 images (727 batches)
   Test:       5,818 images (728 batches)

⚡ Batch: 8, Workers: 0
✅ Ready for training!


In [18]:
# ============================================
# CELL 6: Create Model (EfficientNet-B4)
# ============================================
print("="*70)
print("🤖 CREATING MODEL")
print("="*70)

# Create EfficientNet-B4 model
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n🤖 Model: EfficientNet-B4")
print(f"📊 Classes: {num_classes}")
print(f"🔢 Total Parameters: {total_params:,}")
print(f"🔢 Trainable Parameters: {trainable_params:,}")
print(f"⚙️ Optimizer: AdamW (lr=0.0001)")
print(f"📅 Scheduler: CosineAnnealingLR (10 epochs)")
print(f"🖥️ Device: {device}")

🤖 CREATING MODEL

🤖 Model: EfficientNet-B4
📊 Classes: 34
🔢 Total Parameters: 17,609,578
🔢 Trainable Parameters: 17,609,578
⚙️ Optimizer: AdamW (lr=0.0001)
📅 Scheduler: CosineAnnealingLR (10 epochs)
🖥️ Device: cuda


In [ ]:
# ============================================
# CELL 7: Training Loop (10 Epochs)
# ============================================
import time

print("="*70)
print("🚀 STARTING TRAINING (10 Epochs)")
print("="*70)

epochs = 10
best_val_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

# Save directory
SAVE_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\saved_models")
SAVE_DIR.mkdir(parents=True, exist_ok=True)
print(f"💾 Models will be saved to: {SAVE_DIR}")

for epoch in range(epochs):
    start_time = time.time()
    
    # ============ Training Phase ============
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
    for images, labels in pbar:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        if USE_AMP:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*train_correct/train_total:.1f}%'
        })
    
    train_acc = 100. * train_correct / train_total
    train_loss = train_loss / len(train_loader)
    
    # ============ Validation Phase ============
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            if USE_AMP:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * val_correct / val_total
    val_loss = val_loss / len(val_loader)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'class_names': class_names,
            'num_classes': num_classes,
            'history': history
        }
        torch.save(checkpoint, SAVE_DIR / 'pakistan_model_best.pth')
        print(f"   💾 New best model saved! (Val Acc: {val_acc:.2f}%)")
    
    # Save class names
    with open(SAVE_DIR / 'class_names.json', 'w') as f:
        json.dump(class_names, f, indent=2)
    
    scheduler.step()
    
    elapsed = time.time() - start_time
    eta = (epochs - epoch - 1) * elapsed / 60
    
    print(f"\n📊 Epoch {epoch+1}/{epochs}:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"   Best Val Acc: {best_val_acc:.2f}%")
    print(f"   ⏱️ Time: {elapsed:.0f}s | ETA: {eta:.1f} min")
    print("-"*70)

print(f"\n" + "="*70)
print(f"✅ TRAINING COMPLETE!")
print(f"🏆 Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"💾 Model saved to: {SAVE_DIR / 'pakistan_model_best.pth'}")
print("="*70)

🚀 STARTING TRAINING (10 Epochs)
💾 Models will be saved to: D:\kisaan madadgaar\Plant-Disease-Detection\saved_models


Epoch 1/10 [Val]: 100%|██████████| 727/727 [11:27<00:00,  1.06it/s]


   💾 New best model saved! (Val Acc: 93.67%)

📊 Epoch 1/10:
   Train Loss: 0.6148 | Train Acc: 80.75%
   Val Loss:   0.1746 | Val Acc:   93.67%
   Best Val Acc: 93.67%
   ⏱️ Time: 9797s | ETA: 1469.6 min
----------------------------------------------------------------------


Epoch 2/10 [Val]: 100%|██████████| 727/727 [07:32<00:00,  1.61it/s]


   💾 New best model saved! (Val Acc: 96.29%)

📊 Epoch 2/10:
   Train Loss: 0.1775 | Train Acc: 93.72%
   Val Loss:   0.1040 | Val Acc:   96.29%
   Best Val Acc: 96.29%
   ⏱️ Time: 8558s | ETA: 1141.1 min
----------------------------------------------------------------------


...

Epoch 8/10 [Val]: 100%|██████████| 727/727 [06:33<00:00,  1.85it/s]



📊 Epoch 8/10:
   Train Loss: 0.0355 | Train Acc: 98.74%
   Val Loss:   0.1249 | Val Acc:   97.47%
   Best Val Acc: 97.82%
   ⏱️ Time: 7954s | ETA: 265.1 min
----------------------------------------------------------------------


Epoch 9/10 [Val]: 100%|██████████| 727/727 [04:23<00:00,  2.75it/s]



📊 Epoch 9/10:
   Train Loss: 0.0322 | Train Acc: 98.87%
   Val Loss:   0.0686 | Val Acc:   97.80%
   Best Val Acc: 97.82%
   ⏱️ Time: 12175s | ETA: 202.9 min
----------------------------------------------------------------------


Epoch 10/10 [Train]:  47%|████▋     | 1589/3394 [28:12<29:05,  1.03it/s, loss=0.0026, acc=99.0%]  

In [2]:
# ============================================
# CELL 8: Test Evaluation
# ============================================
print("="*70)
print("🧪 EVALUATING ON TEST SET")
print("="*70)

# Load best model
checkpoint = torch.load(SAVE_DIR / 'pakistan_model_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

test_correct = 0
test_total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)
        
        if USE_AMP:
            with autocast():
                outputs = model(images)
        else:
            outputs = model(images)
        
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = 100. * test_correct / test_total

print(f"\n" + "="*70)
print(f"📊 FINAL TEST RESULTS")
print("="*70)
print(f"\n🎯 Test Accuracy: {test_acc:.2f}%")
print(f"✅ Correct: {test_correct:,} / {test_total:,}")
print("="*70)

🧪 EVALUATING ON TEST SET


NameError: name 'torch' is not defined

In [6]:
# ============================================
# QUICK TEST - Load saved model and check
# ============================================
import json
from pathlib import Path
import torch

SAVE_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\saved_models")

# Check if model exists
if (SAVE_DIR / 'pakistan_model_best.pth').exists():
    print("✅ Model file found!")
    
    # Load checkpoint
    checkpoint = torch.load(SAVE_DIR / 'pakistan_model_best.pth', map_location='cpu')
    
    print(f"\n📊 LOCAL TRAINING RESULTS:")
    print("="*50)
    print(f"🏆 Best Val Accuracy: {checkpoint['val_acc']:.2f}%")
    print(f"📊 Classes: {checkpoint['num_classes']}")
    print(f"📈 Epochs: {checkpoint['epoch'] + 1}")
    print(f"\n📋 Class Names:")
    for i, name in enumerate(checkpoint['class_names']):
        print(f"   {i+1}. {name}")
    print("="*50)
    
    # Training history
    if 'history' in checkpoint:
        h = checkpoint['history']
        print(f"\n📈 Final Train Acc: {h['train_acc'][-1]:.2f}%")
        print(f"📈 Final Val Acc: {h['val_acc'][-1]:.2f}%")
else:
    print("❌ Model not found - run training first!")

✅ Model file found!

📊 LOCAL TRAINING RESULTS:
🏆 Best Val Accuracy: 97.82%
📊 Classes: 34
📈 Epochs: 6

📋 Class Names:
   1. Cotton___diseased_cotton_leaf
   2. Cotton___diseased_cotton_plant
   3. Cotton___fresh_cotton_leaf
   4. Cotton___fresh_cotton_plant
   5. Mango___Anthracnose
   6. Mango___Bacterial_Canker
   7. Mango___Cutting_Weevil
   8. Mango___Die_Back
   9. Mango___Gall_Midge
   10. Mango___Healthy
   11. Mango___Powdery_Mildew
   12. Mango___Sooty_Mould
   13. Plantvillage___Pepper__bell___Bacterial_spot
   14. Plantvillage___Pepper__bell___healthy
   15. Plantvillage___Potato___Early_blight
   16. Plantvillage___Potato___Late_blight
   17. Plantvillage___Potato___healthy
   18. Plantvillage___Tomato_Bacterial_spot
   19. Plantvillage___Tomato_Early_blight
   20. Plantvillage___Tomato_Late_blight
   21. Plantvillage___Tomato_Leaf_Mold
   22. Plantvillage___Tomato_Septoria_leaf_spot
   23. Plantvillage___Tomato_Spider_mites_Two_spotted_spider_mite
   24. Plantvillage___Toma

In [7]:
# ============================================
# 🔥 ENSEMBLE MODEL - Random Forest on Both Models
# ============================================
import torch
import timm
import json
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm

print("="*70)
print("🔥 ENSEMBLE MODEL SETUP")
print("="*70)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {device}")

# Paths
LOCAL_MODEL_PATH = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\saved_models\pakistan_model_best.pth")
COLAB_MODEL_PATH = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\Flask Deployed App\pakistan_model_best.pth")

# Check which models exist
print(f"\n📁 Model Files:")
print(f"   Local Model: {'✅ Found' if LOCAL_MODEL_PATH.exists() else '❌ Not Found'}")
print(f"   Colab Model: {'✅ Found' if COLAB_MODEL_PATH.exists() else '❌ Not Found'}")

# Load Local Model (34 classes - Pakistani crops)
print("\n🔄 Loading Local Model (34 classes)...")
local_checkpoint = torch.load(LOCAL_MODEL_PATH, map_location=device)
local_classes = local_checkpoint['class_names']
local_num_classes = local_checkpoint['num_classes']

local_model = timm.create_model('efficientnet_b4', pretrained=False, num_classes=local_num_classes)
local_model.load_state_dict(local_checkpoint['model_state_dict'])
local_model = local_model.to(device)
local_model.eval()
print(f"   ✅ Local Model: {local_num_classes} classes")

# Load Colab Model (38 classes - PlantVillage)
print("\n🔄 Loading Colab Model (38 classes)...")
colab_checkpoint = torch.load(COLAB_MODEL_PATH, map_location=device)
colab_classes = colab_checkpoint['class_names'] if 'class_names' in colab_checkpoint else None

# Try loading class names from JSON
if colab_classes is None:
    colab_json = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\Flask Deployed App\class_names.json")
    if colab_json.exists():
        with open(colab_json, 'r') as f:
            colab_classes = json.load(f)

colab_num_classes = len(colab_classes) if colab_classes else 38

colab_model = timm.create_model('efficientnet_b4', pretrained=False, num_classes=colab_num_classes)
colab_model.load_state_dict(colab_checkpoint['model_state_dict'])
colab_model = colab_model.to(device)
colab_model.eval()
print(f"   ✅ Colab Model: {colab_num_classes} classes")

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("\n" + "="*70)
print("✅ BOTH MODELS LOADED!")
print("="*70)
print(f"\n📊 Local Model Classes ({local_num_classes}):")
for i, c in enumerate(local_classes[:10]):
    print(f"   {i+1}. {c}")
print("   ...")

print(f"\n📊 Colab Model Classes ({colab_num_classes}):")
for i, c in enumerate(colab_classes[:10]):
    print(f"   {i+1}. {c}")
print("   ...")

🔥 ENSEMBLE MODEL SETUP
🖥️ Device: cuda

📁 Model Files:
   Local Model: ✅ Found
   Colab Model: ✅ Found

🔄 Loading Local Model (34 classes)...
   ✅ Local Model: 34 classes

🔄 Loading Colab Model (38 classes)...
   ✅ Colab Model: 38 classes

✅ BOTH MODELS LOADED!

📊 Local Model Classes (34):
   1. Cotton___diseased_cotton_leaf
   2. Cotton___diseased_cotton_plant
   3. Cotton___fresh_cotton_leaf
   4. Cotton___fresh_cotton_plant
   5. Mango___Anthracnose
   6. Mango___Bacterial_Canker
   7. Mango___Cutting_Weevil
   8. Mango___Die_Back
   9. Mango___Gall_Midge
   10. Mango___Healthy
   ...

📊 Colab Model Classes (38):
   1. Apple___Apple_scab
   2. Apple___Black_rot
   3. Apple___Cedar_apple_rust
   4. Apple___healthy
   5. Blueberry___healthy
   6. Cherry_(including_sour)___Powdery_mildew
   7. Cherry_(including_sour)___healthy
   8. Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
   9. Corn_(maize)___Common_rust_
   10. Corn_(maize)___Northern_Leaf_Blight
   ...


In [8]:
# ============================================
# 🎯 ENSEMBLE PREDICTION FUNCTION
# ============================================

def get_ensemble_features(image_path):
    """Get predictions from both models as features for Random Forest"""
    
    # Load and transform image
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        # Local model predictions (probabilities)
        local_output = local_model(input_tensor)
        local_probs = torch.softmax(local_output, dim=1).cpu().numpy().flatten()
        
        # Colab model predictions (probabilities)
        colab_output = colab_model(input_tensor)
        colab_probs = torch.softmax(colab_output, dim=1).cpu().numpy().flatten()
    
    # Combine features: [local_probs (34), colab_probs (38), top predictions]
    local_top = local_probs.max()
    local_pred = local_probs.argmax()
    colab_top = colab_probs.max()
    colab_pred = colab_probs.argmax()
    
    # Feature vector
    features = np.concatenate([
        local_probs,           # 34 features
        colab_probs,           # 38 features  
        [local_top, colab_top, local_pred, colab_pred]  # 4 extra features
    ])
    
    return features, local_pred, colab_pred, local_top, colab_top

def predict_single(image_path):
    """Predict using both models and show results"""
    features, local_pred, colab_pred, local_conf, colab_conf = get_ensemble_features(image_path)
    
    print(f"\n📷 Image: {Path(image_path).name}")
    print("-"*50)
    print(f"🔵 Local Model:  {local_classes[local_pred]} ({local_conf*100:.1f}%)")
    print(f"🟢 Colab Model:  {colab_classes[colab_pred]} ({colab_conf*100:.1f}%)")
    
    # Simple voting - higher confidence wins
    if local_conf > colab_conf:
        final = local_classes[local_pred]
        final_conf = local_conf
        source = "Local"
    else:
        final = colab_classes[colab_pred]
        final_conf = colab_conf
        source = "Colab"
    
    print(f"🏆 Final (Max Conf): {final} ({final_conf*100:.1f}%) - from {source}")
    
    return features, final

print("✅ Ensemble prediction function ready!")
print("\n📝 Usage: predict_single('path/to/image.jpg')")

✅ Ensemble prediction function ready!

📝 Usage: predict_single('path/to/image.jpg')


In [9]:
# ============================================
# 🌲 TRAIN RANDOM FOREST ENSEMBLE
# ============================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import os

print("="*70)
print("🌲 TRAINING RANDOM FOREST ENSEMBLE")
print("="*70)

# Collect data from test images
DATA_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\data\downloads\PakistanCrops_Merged")

# Build features and labels
print("\n🔄 Extracting features from both models...")

X_features = []
y_labels = []
all_class_names = []

# Get all class folders
class_folders = sorted([f for f in DATA_DIR.iterdir() if f.is_dir()])

for class_idx, folder in enumerate(tqdm(class_folders, desc="Processing classes")):
    class_name = folder.name
    all_class_names.append(class_name)
    
    # Get images (sample 20 per class for speed)
    images = list(folder.glob('*.jpg')) + list(folder.glob('*.JPG')) + list(folder.glob('*.png'))
    sample_images = images[:20]  # Take first 20
    
    for img_path in sample_images:
        try:
            features, _, _, _, _ = get_ensemble_features(str(img_path))
            X_features.append(features)
            y_labels.append(class_idx)
        except Exception as e:
            continue

X_features = np.array(X_features)
y_labels = np.array(y_labels)

print(f"\n✅ Collected {len(X_features)} samples from {len(all_class_names)} classes")
print(f"📊 Feature shape: {X_features.shape}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

print(f"\n📊 Train: {len(X_train)} samples")
print(f"📊 Test: {len(X_test)} samples")

# Train Random Forest
print("\n🌲 Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n" + "="*70)
print(f"🎯 RANDOM FOREST ENSEMBLE RESULTS")
print("="*70)
print(f"\n🏆 Ensemble Accuracy: {accuracy*100:.2f}%")
print(f"\n📊 Comparison:")
print(f"   Local Model (EfficientNet):  97.82%")
print(f"   Colab Model (EfficientNet):  99.26%")
print(f"   🌲 Random Forest Ensemble:   {accuracy*100:.2f}%")

🌲 TRAINING RANDOM FOREST ENSEMBLE

🔄 Extracting features from both models...


Processing classes: 100%|██████████| 34/34 [02:16<00:00,  4.01s/it]



✅ Collected 680 samples from 34 classes
📊 Feature shape: (680, 76)

📊 Train: 544 samples
📊 Test: 136 samples

🌲 Training Random Forest...

🎯 RANDOM FOREST ENSEMBLE RESULTS

🏆 Ensemble Accuracy: 100.00%

📊 Comparison:
   Local Model (EfficientNet):  97.82%
   Colab Model (EfficientNet):  99.26%
   🌲 Random Forest Ensemble:   100.00%


In [10]:
# ============================================
# 💾 SAVE ENSEMBLE MODEL
# ============================================
import joblib

SAVE_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\saved_models")

# Save Random Forest model
rf_path = SAVE_DIR / 'ensemble_rf_model.joblib'
joblib.dump(rf_model, rf_path)

# Save class names
ensemble_info = {
    'ensemble_accuracy': accuracy * 100,
    'local_model_classes': local_classes,
    'colab_model_classes': colab_classes,
    'all_class_names': all_class_names,
    'num_classes': len(all_class_names),
    'feature_size': X_features.shape[1],
    'train_samples': len(X_train),
    'test_samples': len(X_test)
}

with open(SAVE_DIR / 'ensemble_info.json', 'w') as f:
    json.dump(ensemble_info, f, indent=2)

print("="*70)
print("💾 ENSEMBLE MODEL SAVED!")
print("="*70)
print(f"\n📁 Files saved:")
print(f"   ✅ ensemble_rf_model.joblib ({rf_path.stat().st_size/1024:.1f} KB)")
print(f"   ✅ ensemble_info.json")

print(f"\n🎯 Final Ensemble Accuracy: {accuracy*100:.2f}%")
print("\n" + "="*70)
print("🎉 ENSEMBLE COMPLETE!")
print("="*70)

💾 ENSEMBLE MODEL SAVED!

📁 Files saved:
   ✅ ensemble_rf_model.joblib (2530.6 KB)
   ✅ ensemble_info.json

🎯 Final Ensemble Accuracy: 100.00%

🎉 ENSEMBLE COMPLETE!


In [ ]:
# ============================================
# CELL 9: Plot Training History
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history['train_loss']) + 1)

# Loss plot
axes[0].plot(epochs_range, history['train_loss'], 'b-', label='Train Loss', linewidth=2, marker='o')
axes[0].plot(epochs_range, history['val_loss'], 'r-', label='Val Loss', linewidth=2, marker='s')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs_range, history['train_acc'], 'b-', label='Train Accuracy', linewidth=2, marker='o')
axes[1].plot(epochs_range, history['val_acc'], 'r-', label='Val Accuracy', linewidth=2, marker='s')
axes[1].axhline(y=test_acc, color='g', linestyle='--', linewidth=2, label=f'Test Acc: {test_acc:.1f}%')
axes[1].axhline(y=best_val_acc, color='orange', linestyle=':', linewidth=2, label=f'Best Val: {best_val_acc:.1f}%')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(SAVE_DIR / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Plot saved to: {SAVE_DIR / 'training_history.png'}")

In [ ]:
# ============================================
# CELL 10: Save Final Model & Metadata
# ============================================
print("="*70)
print("💾 SAVING FINAL MODEL")
print("="*70)

# Save model info
model_info = {
    'class_names': class_names,
    'num_classes': num_classes,
    'test_accuracy': test_acc,
    'best_val_accuracy': best_val_acc,
    'model_architecture': 'efficientnet_b4',
    'total_images': len(all_image_paths),
    'epochs_trained': len(history['train_loss']),
    'final_train_acc': history['train_acc'][-1],
    'final_val_acc': history['val_acc'][-1],
    'device': str(device)
}

with open(SAVE_DIR / 'model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

with open(SAVE_DIR / 'training_history.json', 'w') as f:
    json.dump(history, f, indent=2)

# Copy to Flask app folder
FLASK_DIR = Path(r"D:\kisaan madadgaar\Plant-Disease-Detection\Flask Deployed App")
if FLASK_DIR.exists():
    import shutil
    shutil.copy(SAVE_DIR / 'pakistan_model_best.pth', FLASK_DIR / 'pakistan_model_best.pth')
    shutil.copy(SAVE_DIR / 'class_names.json', FLASK_DIR / 'class_names.json')
    print(f"\n📁 Copied model files to Flask app folder!")

print(f"\n💾 Files saved:")
print(f"   ✅ pakistan_model_best.pth (model weights)")
print(f"   ✅ class_names.json (class labels)")
print(f"   ✅ model_info.json (metadata)")
print(f"   ✅ training_history.json (training log)")
print(f"   ✅ training_history.png (plot)")

# File sizes
model_size = (SAVE_DIR / 'pakistan_model_best.pth').stat().st_size / (1024*1024)
print(f"\n📊 Model size: {model_size:.1f} MB")

print(f"\n" + "="*70)
print("🎉 ALL DONE!")
print("="*70)
print(f"\n📁 Model saved to: {SAVE_DIR}")
print(f"🎯 Test Accuracy: {test_acc:.2f}%")
print(f"🏆 Best Val Accuracy: {best_val_acc:.2f}%")
print(f"\n🚀 Next: Run Flask app to test the model!")

## ✅ Training Complete!

### 📁 Saved Files:
- `saved_models/pakistan_model_best.pth` - Model weights
- `saved_models/class_names.json` - Class labels
- `saved_models/model_info.json` - Metadata
- `saved_models/training_history.png` - Training plot

### 🚀 Next Steps:
1. Model files automatically copied to `Flask Deployed App/` folder
2. Run the Flask app: `python app.py`
3. Test with images!

### 🇵🇰 کسان مددگار - Helping Pakistani Farmers with AI!